<h1><center><u> Code Implementation </u></center></h1>

In [ ]:
#importing Libraries
import os
import time
import random
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from keras.models import Model
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from sklearn.model_selection import KFold
from tensorflow.keras.regularizers import L2
from tensorflow.keras.models import Sequential
from keras.layers import Input, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from icecream import ic
from sklearn.metrics import accuracy_score
from sklearn.impute import KNNImputer
from dateutil.relativedelta import relativedelta
#from sklearn.impute import IterativeImputer, SimpleImputer, KNNImputer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.linear_model import BayesianRidge, LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import pearsonr
from sklearn.impute import SimpleImputer
from tensorflow.keras import layers, models, optimizers, regularizers
from tensorflow.keras.callbacks import ModelCheckpoint
# from dhs_preprocessing_functions import *
from pandarallel import pandarallel
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
# initialization
pandarallel.initialize()
from keras import Sequential, layers, regularizers, optimizers
from dhs_modelling_functions_new import final_ds_droping_cols, fold_generator

In [ ]:
#setting up random seeds for reproducibility
tf.random.set_seed(6688)
random.seed(6688)
np.random.seed(6688)

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu,True)
    except RuntimeError as e:
        raise e

# Dataset loading

*** given csv (5_grouped_df_V3_HR_adm2_gaul_joined_with_ipc_all.csv) has no 'GEID; init' column which was creating error with fold_generator function.
so, took reference from https://github.com/gheisenberg/FoodSecurity/tree/main/DHS

In [ ]:
input_dir = "C:/Users/rubai/remote_server_sven_dataset_2024/"
# min_version = 3
overwrite_pqt = True
dataset_type = 'HR' 
urban_rural_all_mode = 'all'
group_by_col = 'adm2_gaul'
out_f = f"{input_dir}5_grouped_df_V3_{dataset_type}_{group_by_col}_joined_with_ipc_{urban_rural_all_mode}.pkl"

# Scale options
scale_numerical_data = False
scale_all_data = True
leave_out_encodings = True
zero_one_scale_categorical = False
scale_labels = True

# More Options
drop_agriculture = False

in_f = f"{input_dir}5_grouped_df_V3_{dataset_type}_{group_by_col}_joined_with_ipc_{urban_rural_all_mode}.pkl"

In [ ]:
input_df_0 = pd.read_pickle(in_f)
#for c in input_df.columns:
    #if 'year' in c or 'Year' in c:
        #print(c)

# if urban_rural_all_mode == 'U':
#     drop_agriculture = True
df = final_ds_droping_cols(input_df_0, drop_meta=True, drop_food_help=True, drop_perc=40,
                           retain_month=False, drop_highly_correlated_cols=False, drop_region=True, 
                 drop_data_sets=['Meta one-hot encoding', 'Meta frequency encoding'], 
                 use_NAN_amount_and_replace_NANs_in_categorical=False, drop_agricultural_cols=drop_agriculture, 
                 drop_below_version=False, numerical_data=['mean'], retain_adm=False, 
                 retain_GEID_init=False, verbose=3)
#for col in df.columns:
    #print(col)

<h1><center> <div class="alert alert-danger"> Regular Autoencoder </div> </center></h1>

# controlled cross validation using fold_generator for simple autoencoder

### Step-by-Step Process 

1. **Step 1:**  **Getting the Data**
   - data has been loaded from <a href="https://github.com/gheisenberg/FoodSecurity/tree/main/DHS" target="_blank">Dataset</a>
   - dataset has been tuned according to our needs using final_ds_droping_cols function
   - temporarily dataset has been imputed using simple imputer (mean, median)
   - in next step, more imputers will be applied 

2. **Step 2:**  **Applying controlled cross validation**
   - used fold_generator function during cross validation
   - only simple autoencoder is analyzed on test_updated_v2

3. **Step 3:**  **Finalized simple AE training and evaluation**
   - during AE training, have not used fold_generator function
   - rmse, r2 score, correlation metrics are evaluated on unseen test data
  
4. **Step 3:**  **Improving the dataset by using transformations**
   - log transformation, square root transformation, box-cox transformation etc. will be applied and analyzed one by one for not-numerical columns
   - dataset is highly skewed (most of the columns), transformations might help mitigating this issue

<a href="https://github.com/gheisenberg/FoodSecurity/blob/main/DHS/dhs_modelling_functions.py" target="_blank">Fold Generator Function</a>

In [ ]:
# rmse function
def root_mean_squared_error(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_pred - y_true)))

In [ ]:
# the simple autoencoder model
def build_autoencoder(input_dim):
    autoencoder = models.Sequential([
        layers.Dense(128, activation='relu', input_dim=input_dim, kernel_initializer='he_uniform', activity_regularizer=regularizers.L2(0.001)),
        layers.Dense(64, activation='relu', kernel_initializer='he_uniform', activity_regularizer=regularizers.L2(0.001)),
        layers.Dense(10, activation='relu', kernel_initializer='he_uniform', name='bottleneck'),
        layers.Dense(64, activation='relu', kernel_initializer='he_uniform'),
        layers.Dense(128, activation='relu', kernel_initializer='he_uniform'),
        layers.Dense(input_dim, activation='linear', kernel_initializer='he_uniform')  
    ])
    return autoencoder

# setting up imputers
imputers = [
    #{'name': 'Simple Mean Imputer', 'imputer': SimpleImputer(strategy='mean')},
    #{'name': 'Simple Median Imputer', 'imputer': SimpleImputer(strategy='median')},
    {'name': 'KNN Imputer', 'imputer': KNNImputer(n_neighbors=5)}, 
    #{'name': 'Random Forest Imputer', 'imputer': IterativeImputer(estimator=RandomForestRegressor(n_estimators=5), max_iter=10, random_state=0)} 
]

numerical_only = df.select_dtypes(include=[np.number])
print((numerical_only)) #for scaling purpose we will take only numerical ones
                       # for now, we will do same scaling for all columns
                       # later on, diff scalers can be used for nums and cat columns


# imputation
for i in imputers:
    model = i['imputer']
    first_imputed = model.fit_transform(numerical_only)
    first_df_imputed = pd.DataFrame(first_imputed, columns=numerical_only.columns)
    print(f"Imputed data using {i['name']}:")
    print(first_df_imputed)

    scale_all=True
    # scaling data if flag is true
    if scale_all:
        scaler = StandardScaler()
        first_df_imputed_scaled = scaler.fit_transform(first_df_imputed)
        print('Scaled data:')
        print(first_df_imputed_scaled) 
        
# scaled data back to df
df_scaled = pd.DataFrame(first_df_imputed_scaled, columns=numerical_only.columns)
print('Scaled data back to df:', df_scaled)

# extracting specific columns
columns_to_extract = ['Meta; GEID_init', 'Meta; adm0_gaul']
df_extracted = df[columns_to_extract]

# adding extracted columns to the beginning of df_scaled
df_scaled = pd.concat([df_extracted.reset_index(drop=True), df_scaled.reset_index(drop=True)], axis=1)
print('joing scaled + geid init and adm0 altogether', df_scaled)
        
input_df=df_scaled.copy() # which will be considered as actual values and input of folds during cross-val
print('input_df=df_scaled as input to fold generator',  input_df)

fold_by = 'survey'
# using fold generator
fold_gen = fold_generator(input_df, fold_by, n_splits=5)

# for 5 folds, iteration=5
for fold, (train_index, test_index) in enumerate(fold_gen):
    X_train, X_test = input_df.loc[train_index], input_df.loc[test_index]

    # dropping categorical and fully avaiable cols which does not need imputation
    dropping_cols = ['Meta; adm0_gaul', 'Meta; GEID_init', 'Meta; year']
    X_train = X_train.drop(columns=dropping_cols)
    X_test = X_test.drop(columns=dropping_cols)

    # compiling the autoencoder model
    autoencoder = build_autoencoder(X_train.shape[1])
    autoencoder.compile(optimizer=optimizers.Adam(learning_rate=0.001), loss='mse', metrics=[root_mean_squared_error])

    es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=30)
    history = autoencoder.fit(X_train, X_train, epochs=30, batch_size=32, shuffle=True, callbacks=[es], validation_data=(X_test, X_test))
    X_pred = autoencoder.predict(X_test)
    print('X_pred', X_pred)
    print('X_test', X_test)

    # to calculate metrics
    def calculate_metrics(X_test, X_pred):
        if isinstance(X_test, pd.DataFrame):
            index_method = lambda x, i: x.iloc[:, i]  
        else:
            index_method = lambda x, i: x[:, i]       

        for i in range(X_test.shape[1]):
            mse = np.mean((index_method(X_test, i) - index_method(X_pred, i)) ** 2)
            rmse = np.sqrt(mse)
            r2 = r2_score(index_method(X_test, i), index_method(X_pred, i))
            corr, _ = pearsonr(index_method(X_test, i), index_method(X_pred, i))
            print(f"Column {i+1} - MSE: {mse}, RMSE: {rmse}, R2: {r2}, Correlation: {corr}")

    def get_column(data, idx):
        if isinstance(data, pd.DataFrame):
            return data.iloc[:, idx]
        elif isinstance(data, np.ndarray):
            return data[:, idx]

    num_features = X_test.shape[1] 

    # setting the figure and axes for a grid of plots
    cols = 3  # number of columns in subplot grid
    rows = (num_features + cols - 1) // cols  # rows needed

    fig, axes = plt.subplots(rows, cols, figsize=(20, rows * 4 + 2))  
    fig.subplots_adjust(hspace=0.5, wspace=0.3, top=0.95)

    for i in range(num_features):
        ax = axes.flatten()[i]  # flattening the axes array and access by single index
        actual = get_column(X_test, i)
        predicted = get_column(X_pred, i)

        # pearson correlation, tmse, and r2 for each feature
        corr, _ = pearsonr(actual, predicted)
        mse = mean_squared_error(actual, predicted)
        rmse = np.sqrt(mse)
        r2 = r2_score(actual, predicted)

        column_name = X_test.columns[i] if isinstance(X_test, pd.DataFrame) else f"Feature {i}"
        ax.scatter(actual, predicted, alpha=0.5)
        ax.plot([actual.min(), actual.max()], [actual.min(), actual.max()], 'r--') 
        ax.set_xlabel('Actual Values')
        ax.set_ylabel('Predicted Values')
        ax.set_title(column_name) 

        # plotting rmse, r2, and correlation above the plot
        ax.text(0.8, .25, f'R²: {r2:.2f}\nRMSE: {rmse:.2f}\nCorrelation: {corr:.2f}', verticalalignment='top', horizontalalignment='left', transform=ax.transAxes, fontsize=10, bbox=dict(boxstyle="round,pad=0.3", edgecolor='gray', facecolor='none'))

    for j in range(i + 1, rows * cols):
        axes.flatten()[j].axis('off')

    plt.show()

# Final AE training

In [ ]:
df1=df_scaled.copy()
#print('df1',  df1)

# dropping categorical and fully avaiable cols which does not need imputation
dropping_cols = ['Meta; adm0_gaul', 'Meta; GEID_init', 'Meta; year']
df1 = df1.drop(columns=dropping_cols)
print('df1',  df1)

#splitting the data into train, test, and validation sets
train1, test1 = train_test_split(df1, test_size=0.2, random_state=42)
print('train1',  df1)
print('test1',  df1)
train1, val1= train_test_split(train1, test_size=0.2, random_state=42)
actual_ae1=test1.copy()
        
# rmse function
def root_mean_squared_error(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_pred - y_true)))

# autoencoder final model training
# train1 is already imputed with simple imputers and now it's fully completed without any NaNs
input_dim = train1.shape[1]
final_ae = keras.Sequential([
    layers.Dense(128, activation='relu', input_dim=input_dim, kernel_initializer='he_uniform', activity_regularizer=regularizers.L2(0.001)),
    layers.Dense(64, activation='relu', kernel_initializer='he_uniform', activity_regularizer=regularizers.L2(0.001)),
    layers.Dense(10, activation='relu', kernel_initializer='he_uniform', name='bottleneck'), 
    layers.Dense(64, activation='relu', kernel_initializer='he_uniform'),
    layers.Dense(128, activation='relu', kernel_initializer='he_uniform'),
    layers.Dense(input_dim, activation='relu', kernel_initializer='he_uniform')
])

# compiling the model with the same optimizer and loss function
final_ae.compile(optimizer=optimizers.Adam(learning_rate=0.001), loss='mse', metrics=[root_mean_squared_error])
#final_ae.summary()

missing_ratio=0.3

# function to introduce missingness
def introduce_missingness(data, missing_ratio):
    mask = np.random.rand(*data.shape) < missing_ratio
    data[mask] = np.nan
    return data

# introducing missingness at 0.3 for test dataset
test_data_with_missing = introduce_missingness(test1.copy(), 0.3)
test_data_imputed = test_data_with_missing.fillna(method='ffill').fillna(method='bfill')

# training the model
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=30)
history = final_ae.fit(train1, train1, epochs=20, batch_size=32, shuffle=True, callbacks=[es], validation_split=0.2, verbose=1)

# evaluating the model with rmse
predicted_data = final_ae.predict(test_data_imputed) #test_data_imputed=actual data which is unseen test data
mse = np.mean((test_data_imputed - predicted_data) ** 2) # actual data minus imputed data
rmse = np.sqrt(mse)
#print(f"Final RMSE: {rmse}")
X_pred=predicted_data
#print('X_pred', X_pred)
X_test=test_data_imputed
#print('X_test', X_test)

# calculating metrics
def calculate_metrics(X_test, X_pred):
    if isinstance(X_test, pd.DataFrame):
        index_method = lambda x, i: x.iloc[:, i]  
    else:
        index_method = lambda x, i: x[:, i]       

    for i in range(X_test.shape[1]):
        mse = np.mean((index_method(X_test, i) - index_method(X_pred, i)) ** 2)
        rmse = np.sqrt(mse)
        r2 = r2_score(index_method(X_test, i), index_method(X_pred, i))
        corr, _ = pearsonr(index_method(X_test, i), index_method(X_pred, i))

        print(f"Column {i+1} - MSE: {mse}, RMSE: {rmse}, R2: {r2}, Correlation: {corr}")

def get_column(data, idx):
    if isinstance(data, pd.DataFrame):
        return data.iloc[:, idx]
    elif isinstance(data, np.ndarray):
        return data[:, idx]

num_features = X_test.shape[1] 

# setting the figure and axes for a grid of plots
cols = 3  # number of columns in subplot grid
rows = (num_features + cols - 1) // cols  # rows needed

fig, axes = plt.subplots(rows, cols, figsize=(20, rows * 4 + 2))  
fig.subplots_adjust(hspace=0.5, wspace=0.3, top=0.95)

for i in range(num_features):
    ax = axes.flatten()[i]  # flattening the axes array and access by single index
    actual = get_column(X_test, i)
    predicted = get_column(X_pred, i)

    # pearson correlation, rmse, r2 for each feature
    corr, _ = pearsonr(actual, predicted)
    mse = mean_squared_error(actual, predicted)
    rmse = np.sqrt(mse)
    r2 = r2_score(actual, predicted)

    column_name = numerical_only.columns[i] if isinstance(X_test, pd.DataFrame) else f"Feature {i}"
    ax.scatter(actual, predicted, alpha=0.5)
    ax.plot([actual.min(), actual.max()], [actual.min(), actual.max()], 'r--') 
    ax.set_xlabel('Actual Values')
    ax.set_ylabel('Predicted Values')
    ax.set_title(column_name) 

    # plotting rmse, r2 score, and correlation above the plot
    ax.text(0.8, .25, f'R²: {r2:.2f}\nRMSE: {rmse:.2f}\nCorrelation: {corr:.2f}', verticalalignment='top', horizontalalignment='left', transform=ax.transAxes, fontsize=10, bbox=dict(boxstyle="round,pad=0.3", edgecolor='gray', facecolor='none'))

for j in range(i + 1, rows * cols):
    axes.flatten()[j].axis('off')

plt.show()

# Comments:
1. r2=1 denotes to the situation when model is best at predicting the dependant variable (or doing correct imputation on unseen data).
In our case, the negative r2 scores for several columns have to be improved near to 1. More improvements have to be done gradually.

<h1><center> <div class="alert alert-success"> De-noising Autoencoder </div> </center></h1>


<h1><center> <div class="alert alert-warning"> Variational Autoencoder </div> </center></h1>

# 